# Audio Storage System (Access → Oracle Migration)
This notebook demonstrates an end-to-end workflow:
- ingest a flat Access-style export
- clean inconsistent text fields
- normalize into lookup tables (3NF-style)
- run EDA and train a baseline model to estimate storage cost


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_csv('../data/access_export_audio_records.csv')
df.head()


In [ ]:
df.shape, df.dtypes


## Data Preprocessing Part 1
Clean the messy department field, standardize categories, and validate obvious ranges.


In [ ]:
# Standardize department
df['department'] = (
    df['department'].astype(str).str.strip().str.replace(r"\s+", " ", regex=True).str.title()
)

# Standardize storage tiers
df['storage_tier'] = df['storage_tier'].astype(str).str.strip().str.title()

# Basic range checks
df = df[(df['duration_sec'] > 0) & (df['duration_sec'] <= 10*3600)]
df = df[(df['file_size_mb'] > 0) & (df['file_size_mb'] < 5000)]

df.head()


In [ ]:
df.isna().sum()


In [ ]:
df.describe(include='all').T.head(12)


## Normalization (3NF-style)
We build lookup tables and replace text keys with IDs (similar to how you'd load into Oracle).


In [ ]:
# Lookup tables
dept_lu = df[['department']].drop_duplicates().sort_values('department').reset_index(drop=True)
dept_lu['dept_id'] = np.arange(1, len(dept_lu)+1)

type_lu = df[['recording_type']].drop_duplicates().sort_values('recording_type').reset_index(drop=True)
type_lu['type_id'] = np.arange(1, len(type_lu)+1)

tier_lu = df[['storage_tier']].drop_duplicates().sort_values('storage_tier').reset_index(drop=True)
tier_lu['tier_id'] = np.arange(1, len(tier_lu)+1)

# Replace text with IDs
norm = df.merge(dept_lu, on='department').merge(type_lu, on='recording_type').merge(tier_lu, on='storage_tier')

audio_files = norm[[
    'record_id','file_name','dept_id','type_id','tier_id',
    'duration_sec','sample_rate_hz','channels','file_size_mb','created_date','created_by',
    'storage_cost_usd_month'
]].copy()

dept_lu.head(), type_lu.head(), tier_lu.head()


In [ ]:
# Save normalized outputs (as if staging for Oracle load)
dept_lu.to_csv('../reports/departments.csv', index=False)
type_lu.to_csv('../reports/recording_types.csv', index=False)
tier_lu.to_csv('../reports/storage_tiers.csv', index=False)
audio_files.to_csv('../reports/audio_files_normalized.csv', index=False)

audio_files.head()


## Exploratory Data Analysis
We inspect distributions and relationships that drive storage cost.


In [ ]:
plt.figure(figsize=(8,4))
sns.histplot(audio_files['storage_cost_usd_month'], bins=40)
plt.title('Distribution of monthly storage cost')
plt.show()


In [ ]:
plt.figure(figsize=(8,4))
sns.scatterplot(data=audio_files.sample(600, random_state=42), x='file_size_mb', y='storage_cost_usd_month')
plt.title('Storage cost vs file size')
plt.show()


In [ ]:
plt.figure(figsize=(10,4))
sns.countplot(x='tier_id', data=audio_files)
plt.title('Count by storage tier (encoded)')
plt.show()


## Encoding + Correlation Heatmap
We use numeric/encoded fields directly for a quick correlation view.


In [ ]:
corr_cols = ['storage_cost_usd_month','duration_sec','sample_rate_hz','channels','file_size_mb','dept_id','type_id','tier_id']
plt.figure(figsize=(9,6))
sns.heatmap(audio_files[corr_cols].corr(), annot=True, cmap='coolwarm')
plt.title('Correlation matrix')
plt.show()


## Outlier Removal + Train/Test Split + Model
Baseline: DecisionTreeRegressor with GridSearchCV


In [ ]:
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

model_df = audio_files.copy()

z = np.abs(stats.zscore(model_df[['duration_sec','file_size_mb','storage_cost_usd_month']]))
model_df = model_df[(z < 3).all(axis=1)]

X = model_df.drop(columns=['storage_cost_usd_month','file_name','created_date','created_by'])
y = model_df['storage_cost_usd_month']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'max_depth': [3,5,7,9],
    'min_samples_split': [2,4,8],
    'min_samples_leaf': [1,2,4],
    'random_state': [42]
}

grid = GridSearchCV(DecisionTreeRegressor(), param_grid=params, cv=5, n_jobs=-1, verbose=0)
grid.fit(X_train, y_train)

best = grid.best_estimator_
best.fit(X_train, y_train)

pred = best.predict(X_test)

print("Best params:", grid.best_params_)
print("R2:", r2_score(y_test, pred))
print("MAE:", mean_absolute_error(y_test, pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, pred)))


In [ ]:
feat_df = pd.DataFrame({'Feature': X.columns, 'Importance': best.feature_importances_}).sort_values('Importance', ascending=False)
feat_df


In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(data=feat_df, x='Importance', y='Feature')
plt.title('Feature Importance')
plt.show()


## Conclusion
This project shows:
- a practical ETL cleaning step (messy text fields)
- normalization for database loading (3NF-style)
- baseline cost modeling and feature importance to explain drivers
